In [2]:

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import pandas as pd
from googlesearch import search
import requests
from bs4 import BeautifulSoup


def get_espncricinfo_link(player_name):

    if player_name == "Andre Siddharth":
        player_name = "Andre Siddarth"

    query = f"{player_name} ESPNcricinfo"
    try:
        for link in search(query, sleep_interval=5, num_results=5, lang="en"):
            if "espncricinfo.com" in link:
                return link
            else:
                print(f"Could not find ESPNcricinfo link for {player_name}.")
                return None

    except Exception as e:
        print(f"Error searching for {player_name}: {e}. Please try again.")
        return None
    
def get_espncricinfo_number(link):
    return int(link.split("-")[-1]) if link else None

data_dir = "../data/processed"
csv_file_path = f'{data_dir}/IPL_squad.csv'  # Replace with your input CSV file path
output_csv_path = f'{data_dir}/all_teams_squad_with_id.csv'  # Replace with your desired output CSV file path
df = pd.read_csv(csv_file_path)

# with ProcessPoolExecutor(max_workers=1) as executor:
#     df["ESPNcricinfo Link"] = list(tqdm(executor.map(get_espncricinfo_link, df["Player Name"]), total=len(df["Player Name"]), desc="Adding ESPNcricinfo Links"))

df["ESPNcricinfo Link"] = df["Player Name"].apply(get_espncricinfo_link)
df['key_cricinfo'] = df['ESPNcricinfo Link'].apply(get_espncricinfo_number)

print(df[["Player Name", "key_cricinfo"]])

all_players_df = pd.read_csv("../data/raw/cricsheet/people.csv")
players_df = pd.merge(df, all_players_df[["identifier", "key_cricinfo", "name"]], on="key_cricinfo", how="left")

players_df.to_csv(output_csv_path, index=False)

           Player Name  key_cricinfo
0           Vansh Bedi       1413379
1      Andre Siddharth       1440190
2    Ramakrishna Ghosh       1339053
3        Shaik Rasheed       1292497
4     Gurjapneet Singh       1269869
..                 ...           ...
223         Adam Zampa        379504
224      Harshal Patel        390481
225        Pat Cummins        489889
226     Mohammed Shami        481896
227     Jaydev Unadkat        390484

[228 rows x 2 columns]


In [ ]:

        # header = {
        #     "accept": "*/*",
        #     "accept-encoding": "gzip, deflate, br",
        #     "accept-language": "en-US,en;q=0.9",
        #     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        #     # 'User-agent': 'your bot 0.1'
        # }
        # response = requests.get(f"https://www.google.com/search?q={query}")
        # if response.status_code == 200:

        #     soup = BeautifulSoup(response.text, 'html.parser')
        #     links = soup.find_all('a', href=lambda href: href and "espncricinfo" in href)
        
        # else:
        #     print(f"Could not find ESPNcricinfo link for {player_name}.")
        #     return None